In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)
pd.set_option("display.float_format", "{:.2f}".format)
pd.set_option("display.precision", 3)
pd.set_option("display.max_colwidth", None)

## Leitura e primeiras colunas deletadas

In [3]:
entradas = pd.read_csv(r"ENTRADASEBZMORUMBI2025 - Entradas.csv")
entradas = entradas.dropna(subset=("Colaborador"))
entradas.drop(columns=["Unnamed: 49", "Unnamed: 50"], inplace=True)

In [4]:
entradas.columns = [col.lower() for col in entradas.columns]
entradas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1217 entries, 0 to 1216
Data columns (total 52 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   data                           1217 non-null   object 
 1   id                             1217 non-null   float64
 2   colaborador                    1217 non-null   object 
 3   cliente                        1217 non-null   object 
 4   servico                        1217 non-null   object 
 5   valordoservico                 1217 non-null   object 
 6   clientenovo                    1217 non-null   object 
 7   local                          1217 non-null   object 
 8   produto                        1217 non-null   object 
 9   quantidadeproduto              1217 non-null   object 
 10  valortotaldosproduto           1217 non-null   object 
 11  auxvalortotalcompraprodutos    1217 non-null   object 
 12  auxdescontoprodutos            1217 non-null   object

### Definindo classe e funções de limpeza

In [5]:
# file: data_cleaner.py

from typing import Dict, Union
import pandas as pd
import numpy as np


class DataCleaner:
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()

    def to_date(self, date_column: str, date_format: str = "%d/%m/%Y") -> "DataCleaner":
        """Convert a column to datetime."""
        if date_column in self.df.columns:
            self.df[date_column] = pd.to_datetime(
                self.df[date_column], format=date_format, errors="coerce"
            )
        return self

    def normalize_dollar(self, dollar_column: str) -> "DataCleaner":
        """Normalize dollar values: remove currency symbols, convert to float."""
        if dollar_column in self.df.columns:
            self.df[dollar_column] = (
                self.df[dollar_column]
                .astype(str)
                .str.replace("R$", "", regex=True)
                .str.replace(",", ".")
                .str.replace(" ", "")
                .str.replace("0.0.0", "0")
            )
            self.df[dollar_column] = pd.to_numeric(
                self.df[dollar_column], errors="coerce"
            )
        return self

    def to_boolean(self, column: str) -> "DataCleaner":
        """Convert 'SIM'/'NÃO' values to boolean."""
        if column in self.df.columns:
            self.df[column] = (
                self.df[column].map({"NÃO": False, "SIM": True}).astype("boolean")
            )
        return self

    def to_null(self, column: str, null_val: str = "VAZIO") -> "DataCleaner":
        """Convert specific string values to NaN."""
        if column in self.df.columns:
            self.df[column] = self.df[column].replace(null_val, np.nan)
        return self

    def to_its_own_dimension(
        self, key: str, value: str, id_column: str = "id"
    ) -> pd.DataFrame:
        """Extract a normalized dimension table from the current DataFrame."""
        if all(col in self.df.columns for col in [id_column, key, value]):
            new_df = (
                self.df[[id_column, key, value]]
                .drop_duplicates()
                .rename(columns={key: "produtos", value: "quantidade"})
                .assign(type=key)
            )
            return new_df
        else:
            raise ValueError("nem todas ou alguma coluna não presente")

    def get_data(self) -> pd.DataFrame:
        """Return the cleaned DataFrame."""
        return self.df.copy()

In [6]:
class feature_engineering(pd.DataFrame):

    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

    def index_values(self, column: str) -> "feature_engineering":
        """
        Creates a mapping of unique values in a column to unique indices.

        Args:
            column (str): The column to index.

        Returns:
            feature_engineering: The updated DataFrame with an indexed column.
        """
        unique_values = self[column].unique()
        dicionario = {value: key for key, value in enumerate(unique_values)}
        self[f"id_{column}"] = self[column].map(dicionario)
        return self

## Limpando dados

In [7]:
entradas = DataCleaner(entradas)
entradas = entradas.to_date("data")

In [8]:
colunas_valor = (
    [col for col in entradas.df.columns if col.startswith("valor")]
    + [
        "taxamaquina",
        "total(s+p)",
        "total(s+p)*t",
        "total[(s+p)-lp]*t",
        "totals+lp-col",
        "totalcolaborador",
        "colaborador50%",
    ]
    + [col for col in entradas.df.columns if col.startswith("auxvalor")]
    + [col for col in entradas.df.columns if col.startswith("auxdesconto")]
)
for col in colunas_valor:
    entradas = entradas.normalize_dollar(col)

In [9]:
entradas = entradas.to_boolean("clientenovo")

colunas = ["produto", "doces", "salgados", "bebidas"]
for col in colunas:
    entradas = entradas.to_null(col)

## Criando dimensão de produto

In [10]:
produtos = pd.read_csv(
    r"c:\Users\luiz\Documents\GitHub\EZmorumbi\Data Cleaning\ENTRADASEBZMORUMBI2025 - Produtos.csv"
)
produtos.columns = [col.lower() for col in produtos.columns]

In [11]:
dim_servico = produtos[["serviço", "valor do serviço"]].copy()
dim_produto = produtos[["produto", "valor de venda", "valor de compra", "lucro"]].copy()
dim_doce = produtos[["doce", "valor de venda.1", "valor de compra.1", "lucro.1"]].copy()
dim_salgado = produtos[
    ["salgado", "valor de venda.2", "valor de compra.2", "lucro.2"]
].copy()
dim_bebida = produtos[
    ["bebida", "valor de venda.3", "valor de compra.3", "lucro.3"]
].copy()
dim_funcionario = produtos[["funcionario", "porcentagem"]].copy()
dim_taxa = produtos[["funcao", "taxa"]].copy()

In [12]:
dimensoes_produtos = [[dim_produto, dim_doce, dim_salgado, dim_bebida]]

In [13]:
dim_servico.loc[:, ("tipo")] = "servico"
dim_produto.loc[:, ("tipo")] = "barbearia"
dim_doce.loc[:, ("tipo")] = "doce"
dim_salgado.loc[:, ("tipo")] = "salgado"
dim_bebida.loc[:, ("tipo")] = "bebida"
dim_funcionario.loc[:, ("tipo")] = "funcionario"
dim_taxa.loc[:, ("tipo")] = "taxa"

In [14]:
dataframes = [
    dim_servico,
    dim_produto,
    dim_doce,
    dim_salgado,
    dim_bebida,
    dim_funcionario,
    dim_taxa,
]

for data in dataframes:
    data.dropna(inplace=True)

In [15]:
novas_colunas = ["produto", "vl_venda", "vl_compra", "lucro", "tipo"]
dim_produto = novas_colunas
dim_bebida.columns = novas_colunas
dim_doce.columns = novas_colunas
dim_salgado.columns = novas_colunas

dim_servico.rename(columns={"serviço": "servico", "valor do serviço": "vl_venda"})

,servico,vl_venda,tipo
0,Corte,"R$ 60,00",servico
1,Barba,"R$ 50,00",servico
2,Perfil,"R$ 20,00",servico
3,Sobrancelha,"R$ 20,00",servico
4,Higienização Facial,"R$ 50,00",servico
5,Corte+Barba,"R$ 90,00",servico
6,Corte+Sobrancelha,"R$ 80,00",servico
7,Corte+Higienização facial,"R$ 110,00",servico
8,Barba+Sobrancelha,"R$ 70,00",servico
9,Barba+Perfil,"R$ 70,00",servico


In [ ]:
dim_produto_df = pd.DataFrame(dimensoes_produtos[0][0])
dim_produto_df = dim_produto_df.normalize_dollar("lucro")
dim_produto_df = dim_produto_df.normalize_dollar("vl_compra")
dim_produto_df = dim_produto_df.normalize_dollar("vl_venda")

AttributeError: 'list' object has no attribute 'normalize_dollar'

In [ ]:
dim_doce = DataCleaner(dim_doce).normalize_dollar("lucro")

AttributeError: 'DataCleaner' object has no attribute 'copy'

In [ ]:
dim_salgado = dim_salgado.normalize_dollar("lucro")

dim_bebida = dim_bebida.normalize_dollar("lucro")


dim_doce = dim_doce.normalize_dollar("vl_venda")

dim_salgado = dim_salgado.normalize_dollar("vl_venda")

dim_bebida = dim_bebida.normalize_dollar("vl_venda")

dim_doce = dim_doce.normalize_dollar("vl_compra")

dim_salgado = dim_salgado.normalize_dollar("vl_compra")

dim_bebida = dim_bebida.normalize_dollar("vl_compra")


def to_string(dataframe, coluna):

    dataframe[coluna] = dataframe[coluna].astype(str)

    return dataframe[coluna].apply(lambda x: type(x)).value_counts()


to_string(dim_bebida, "produto")

to_string(dim_produto, "produto")


dim_produto = dim_produto[dim_produto["produto"] != "VAZIO"]


dataframes = [dim_bebida, dim_doce, dim_salgado]

for entradas in dataframes:

    entradas.dropna(subset=["vl_venda"], inplace=True)


dim_produto_2 = pd.concat([dim_bebida, dim_doce, dim_salgado])


dim_p_b = dim_produto.merge(
    dim_produto_2, on="produto", how="left", suffixes=("", "n_")
)

dim_p_b.drop(columns=["vl_vendan_", "compran_", "lucron_"], inplace=True)

dim_p_b.dropna(subset="vl_venda", inplace=True)

dim_p_b["tipo"] = dim_p_b["tipo"].apply(lambda x: "barbearia" if pd.isna(x) else x)

dim_p_b.drop_duplicates(subset=["produto"], inplace=True)

dim_p_b = feature_engineering(dim_p_b)

dim_p_b = dim_p_b.index_values("produto")

produto_map = dim_p_b.set_index("produto")["id_produto"].to_dict()

In [ ]:
dim_p_b

In [ ]:
passage: Dict[str, pd.DataFrame] = {}

for key, value in combine_columns.items():
    result_dataframe = data_cleaning.to_its_own_dimension(entradas, key, value)

result_dataframe.dropna(subset="produtos", inplace=True)
result_dataframe = result_dataframe.assign(
    produtos=result_dataframe["produtos"].str.split(",")
).explode("produtos")
result_dataframe["quantidade"] = 1
result_dataframe["produtos"].unique()
fato_produtos = result_dataframe
fato_produtos.rename(columns={"ID": "id_servico"}, inplace=True)
fato_produtos["id_servico"] = fato_produtos["id_servico"].astype(int)
fato_produtos = fato_produtos.merge(
    dim_p_b, left_on="produtos", right_on="produto", how="left"
)

#### Limpeza produtos

## Criando tabela Fato


In [ ]:
tabela_fato = entradas[
    [
        "data",
        "ID",
        "Colaborador",
        "Cliente",
        "Servico",
        "ValordoServico",
        "ClienteNovo",
        "Local",
        "Produto",
        "QuantidadeProduto",
        "ValorTotaldosProduto",
        "Doces",
        "QuantidadeDoces",
        "ValorTotaldosDoces",
        "Salgados",
        "QuantidadeSalgados",
        "Bebidas",
        "QuantidadeBebidas",
        "ValorTotaldasBebidas",
        "FormadePagamento",
        "TaxaMaquina",
        "Total(S+P)",
        "Total(S+P)*T",
        "Total[(S+P)-LP]*T",
        "TotalS+LP-Col",
        "TotalColaborador",
        "Colaborador50%",
    ]
]

tabela_fato = tabela_fato.rename(
    columns={
        "data": "data",
        "ID": "id",
        "Colaborador": "colaborador",
        "Cliente": "cliente",
        "Servico": "servico",
        "ValordoServico": "vl_servico",
        "ClienteNovo": "cliente_novo",
        "Local": "fidelizado",
        "Produto": "produto",
        "QuantidadeProduto": "qtd_produto",
        "ValorTotaldosProduto": "vl_total_produto",
        "Doces": "doce",
        "QuantidadeDoces": "qtd_doce",
        "ValorTotaldosDoces": "vl_total_doces",
        "Salgados": "salgado",
        "QuantidadeSalgados": "qtd_salgado",
        "ValorTotaldosSalgados": "vl_total_salgado",
        "Bebidas": "bebida",
        "QuantidadeBebidas": "qtd_bebida",
        "ValorTotaldasBebidas": "vl_total_bebida",
        "FormadePagamento": "forma_de_pagamento",
        "TaxaMaquina": "tx_maquina",
        "Total(S+P)": "total_sp",
        "Total(S+P)*T": "total_spt",
        "Total[(S+P)-LP]*T": "total_splt",
        "TotalS+LP-Col": "total_slp_col",
        "TotalColaborador": "total_colaborador",
        "Colaborador50%": "colaborador_50",
    }
)

## Criação da tabela 

In [ ]:
import mysql.connector


def create_table(cursor):
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS tabela_fato (
            data DATETIME,
            id DOUBLE,
            colaborador TEXT,
            cliente TEXT,
            servico TEXT,
            vl_servico DOUBLE,
            cliente_novo BOOLEAN,
            fidelizado TEXT,
            produto TEXT,
            qtd_produto TEXT,
            vl_total_produto DOUBLE,
            doce TEXT,
            qtd_doce DOUBLE,
            vl_total_doces DOUBLE,
            salgado TEXT,
            qtd_salgado DOUBLE,
            bebida TEXT,
            qtd_bebida TEXT,
            vl_total_bebida DOUBLE,
            forma_de_pagamento TEXT,
            tx_maquina DOUBLE,
            total_sp DOUBLE,
            total_spt DOUBLE,
            total_splt DOUBLE,
            total_slp_col DOUBLE,
            total_colaborador DOUBLE,
            colaborador_50 DOUBLE
        )
    """
    )


def insert_data(cursor, entradas):
    for _, row in entradas.iterrows():
        cursor.execute(
            """
            INSERT INTO tabela_fato VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
            tuple(row),
        )


def main(entradas):
    conn = mysql.connector.connect(
        host="database-atinova.ct6oomqu6y49.sa-east-1.rds.amazonaws.com",
        user="integrantes",
        password="grupoPI2025",
        database="barbearia",  # Substituir pelo nome correto do banco de dados
        port=3306,
    )
    cursor = conn.cursor()

    create_table(cursor)

    entradas.fillna("", inplace=True)  # Substituir NaN por strings vazias
    insert_data(cursor, entradas)

    conn.commit()
    cursor.close()
    conn.close()


# Chame a função passando seu DataFrame diretamente
main(tabela_fato)